# PyConJP2022
## 公的統計APIデータ取得PythonライブラリJPy-DataReader

In [ ]:
%%HTML
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>

# e-Stat API機能の事前準備

e-Stat API機能に事前に[ユーザー登録](https://www.e-stat.go.jp/mypage/user/preregister)して、アプリケーションIDをを取得します。

In [ ]:
api_key = "Your_Application_ID" # 取得したアプリケーションID　

アプリケーションIDを環境変数に設定しておくと便利です。

In [ ]:
!export ESTAT_API_KEY="Your_Application_ID"

In [ ]:
# 環境変数エクスポート後、再度Notebookの立ち上げが必要
import os
api_key = os.getenv("ESTAT_API_KEY")

# e-Stat API機能でのデータ取得

statsDataIdは「統計表情報取得」で得られる統計表IDです。e-Statの[データベース](https://www.e-stat.go.jp/stat-search/database?page=1)から検索できます。「政府統計コード」ではなく、「統計表表示 ID」です。

## requestsでのデータ取得

まず、JPy-DataReaderを使わず、requestsでデータ取得してみます。

In [ ]:
!pip install requests

In [ ]:
import requests

url = "https://api.e-stat.go.jp/rest/3.0/app/json/getStatsData?"
params = {
    "appId": api_key,
    "statsDataId": "0003000795",
}

res = requests.get(url, params)
out = res.json()

out

JSONの中身を見てみます。

In [ ]:
out.keys()

In [ ]:
out['GET_STATS_DATA'].keys()

In [ ]:
out['GET_STATS_DATA']['STATISTICAL_DATA'].keys()

In [ ]:
out['GET_STATS_DATA']['STATISTICAL_DATA'][ 'DATA_INF'].keys()

統計データ自体は以下に格納されています。

In [ ]:
out['GET_STATS_DATA']['STATISTICAL_DATA'][ 'DATA_INF']['VALUE']

pandasのデータフレーム形式で読み込みます。

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd

df = pd.DataFrame(out['GET_STATS_DATA']['STATISTICAL_DATA'][ 'DATA_INF']['VALUE'])
df

VALUEだけでは、それぞれの列が何を表しているのかわかりません。各列のメタデータはCLASS_INFに格納されており、列ID名で結合することができます。

In [ ]:
out['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']

In [ ]:
pd.DataFrame(out['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ'][1][ 'CLASS'])

値の列「$」を見ると、数値でない欠損と思われる特殊文字があります。

In [ ]:
sorted(df['$'].unique().tolist())

特殊文字はNOTEで、どのような種類があるかとその意味を調べることができます。

In [ ]:
out['GET_STATS_DATA']['STATISTICAL_DATA'][ 'DATA_INF']['NOTE']

# JPy-DataReaderでデータ取得

e-Statなど公的統計APIからデータを取得するPythonのライブラリJPy-DataReaderを作りました。

In [ ]:
!pip install -U jpy-datareader

さくっとデータ取得する場合、以下の２行でデータ取得とデータ加工ができます。

In [ ]:
import jpy_datareader as jdr

df = jdr.get_data_estat_statsdata(api_key, statsDataId="0003000795")
df

以下も同じ挙動です。

In [ ]:
import jpy_datareader.data as web

df = web.DataReader("0003000795", "estat", api_key=api_key)
df

よくデータ取得に使われる[pandas-datareader](https://pandas-datareader.readthedocs.io/en/latest/)に仕様を似せています。

In [ ]:
!pip install pandas-datareader

In [ ]:
import pandas_datareader as pdr
pdr.get_data_fred('GS10').tail(10)  # 10年満期米国債市場金利

In [ ]:
import pandas_datareader.data as web
web.DataReader('TUD', 'oecd')  # 労働組合データ

もう少し細かく設定してデータ取得したい場合には、以下を使います。

In [ ]:
from jpy_datareader import estat

statsdata = estat.StatsDataReader(api_key, statsDataId="0003000795")
df = statsdata.read()